<h4><b>Importação de Bibliotecas</b></h4>

In [1]:
import pandas as pd

import requests
import lzma
import json

<h4><b>Funções utilizadas </b></h4>

In [20]:
"""
    Função reutilizável para carregamento dos dados.
"""
def load_dataset(path, file_type):
    if file_type == 'json':
        return pd.read_json(path)
    elif file_type == 'csv':
        return pd.read_csv(path)
    else:
        return print('Erro ao carregar os dados')
    
def rename_features(dataset, renamed_columns, inplace, logs):
    if logs:
        print('-'*100)
        print('Renomeando features')
    """
    Parâmetro inplace modifica o dataframe ou series diretamente, sem criar um novo objeto. 
    As alterações são efetivadas diretamente no datasframe ou series em memória.
    """
    return dataset.rename(columns=renamed_columns, inplace=inplace)

def remove_features(dataset, features, inplace, logs):
    if logs:
        print('-'*100)
        print('Removendo features do dataset')
        
    return dataset.drop(columns=features, inplace=inplace)
    
def validate_numeric_registers(dataset, feature, logs):
    if logs:
        print('-'*100)
        print('Verificando se existem registros não numéricos')
        print(f'Total de registros: {len(dataset[feature])}')
    
    """
    Parâmetro errors especifica como tratar erros que ocorrem durante a conversão. 
    Quando erros='coerce' é definido, qualquer valor que não possa ser convertido em um valor numérico 
    o tipo será substituído por NaN (não é um número).
    """
    numeric_data = pd.to_numeric(dataset[feature], errors='coerce')
    non_integers = dataset[feature][numeric_data.isna()]
    
    if logs:
        print(f'Total de registros não numéricos: {len(non_integers)}')
        print(f'Total de registros numéricos: {len(numeric_data) - len(non_integers)}')
    
        if len(non_integers) > 0:
            if logs:
                print('Lista registros não numéricos')
            print(non_integers)
        else:
            if logs:
                print('Não existem registros não numéricos')
        
def clean_non_numeric_registers(dataset, feature, logs):
    if logs:
        print('-'*100)
        print('Limpando string X de registros não numéricos')
    return [str(item)[1:] if str(item).startswith('X') else item for item in dataset[feature]]

def convert_object_to_int(dataset, feature, logs):
    if logs:
        print('-'*100)
        print('Convertendo tipo de object para int')
    return pd.to_numeric(dataset[feature], errors='coerce').astype(int)

def checks_duplicate_registers(dataset, feature, logs):
    if logs:
        print('-'*100)
        print('Verificando se existem registros duplicados na feature line_id')
    duplicates = dataset[dataset[feature].duplicated()]
    
    if logs:
        print(f'Existem {len(duplicates)} registros duplicados')

def show_duplicate_registers(dataset, feature, logs):
    if logs:
        print('-'*100)
        print('Mostrando registros duplicados na feature line_id')
    duplicates = dataset[dataset[feature].duplicated()]
    
    if logs:
        if len(duplicates) > 0:
            print(duplicates['line_id'])
        else:
            print('Não existem registros duplicados a serem exibidos')

def sequencing_registers(dataset, feature, logs):
    if logs:
        print('-'*100)
        print('Sequenciando registros pela feature line_id')
    return dataset.sort_values(by=feature).reset_index(drop=True)

def remove_duplicate_registers(dataset, feature, logs):
    if logs:
        print('-'*100)
        print('Removendo registros duplicados')
    return dataset.drop_duplicates(subset=[feature])

<h4><b>Coleta dos Dados</b></h4>
<p>A coleta dos dados é automatizada, deste modo, disparamos uma requisição para cada dataset e o download é efetuado.</p>
<p>São declaradas variáveis que recebem as definições de data inicial, data final, ano, mês. Uma lista é montada com range de dias.</p>
<p>São declaradas variáveis que recebem os caminhos, que serão salvos os datasets.</p>
<p>Datas dos datasets capturados:</p>
<ul>
    <li>Mês de Janeiro de 2023 - iniciando no dia 01 e finalizando do dia 31;</li>    
    <li>Mês de Junho de 2024 - iniciando no dia 05 e finalizando do dia 30.</li>  
</ul>

In [11]:
# 1-31 2023 01
# 5-30 2024 06
INITIAL_DATE = 5
FINAL_DATE = 30
YEAR = '2024'
MONTH = '06'
days_list = [f"{num:02}" for num in list(range(INITIAL_DATE, FINAL_DATE + 1))]
PATH_ORIGINALS_DATASETS = '/media/marcos/500GB/00_datasets/originals/'
PATH_INITIAL_PREPROCESSING_DATASETS = '/media/marcos/500GB/00_datasets/initial_preprocessing/'
PATH_CONCATENATED_DATASETS = '/media/marcos/500GB/00_datasets/concatenated/'
PATH_JOINED_DATASETS = '/media/marcos/500GB/00_datasets/joined/'

In [6]:
for day in days_list:
    print(f'Iniciando download, extração e salvamento dos datasets - data: {YEAR}/{MONTH}/{day}')
    list_datasets_names = [
        f'{YEAR}_{MONTH}_{day}_linhas',
        f'{YEAR}_{MONTH}_{day}_pontosLinha',
        f'{YEAR}_{MONTH}_{day}_tabelaLinha',
        f'{YEAR}_{MONTH}_{day}_trechosItinerarios',
        f'{YEAR}_{MONTH}_{day}_tabelaVeiculo'
    ]
    
    for dataset_name in list_datasets_names:
        url = f"https://dadosabertos.c3sl.ufpr.br/curitibaurbs/{dataset_name}.json.xz"
        response = requests.get(url)
        
        if response.status_code == 200:
            # Extraindo (descomprimindo) conteúdo do response
            decompressed_data = lzma.decompress(response.content)
            
            # Carregando conteúdo extraido em formato JSON
            data = json.loads(decompressed_data.decode('utf-8'))
            
            # Salvando arquivo do dataset, formato JSON, em diretório local
            with open(f'{PATH_ORIGINALS_DATASETS}{dataset_name}.json', 'w', encoding='utf-8') as json_file:
                json.dump(data, json_file, indent=4)
            # print(f"Dataset: {dataset_name} extraído e salvo com sucesso!")
        else:
            print(f"Falha ao extrair e salvar dataset. Status code: {response.status_code}")
    print(f'Finalização de download, extração e salvamento dos datasets - data: {YEAR}/{MONTH}/{day}')
    print('-'*100)

Iniciando download, extração e salvamento dos datasets - data: 2024/06/05
Finalização de download, extração e salvamento dos datasets - data: 2024/06/05
----------------------------------------------------------------------------------------------------
Iniciando download, extração e salvamento dos datasets - data: 2024/06/06
Finalização de download, extração e salvamento dos datasets - data: 2024/06/06
----------------------------------------------------------------------------------------------------
Iniciando download, extração e salvamento dos datasets - data: 2024/06/07
Finalização de download, extração e salvamento dos datasets - data: 2024/06/07
----------------------------------------------------------------------------------------------------
Iniciando download, extração e salvamento dos datasets - data: 2024/06/08
Finalização de download, extração e salvamento dos datasets - data: 2024/06/08
-------------------------------------------------------------------------------------

<h4><b>Carregamento dos datasets</b></h4>
<p>Efetuamos o carregamento dos datasets e submetemos os mesmos as etapas de pré-processamento.</p>

<h4><b>Etapas de pré-processamento</b></h4>
<ul>
    <li>Renomeando features;</li>    
    <li>Validando a feature line_id se ela contém apenas registros numéricos;</li>    
    <li>Na feature line_id durante o pré-processamento, foram encontrados registros não numéricos, contendo a string 'X 'número''. Realizamos procedimento de remover esta string 'X' e deixar somente o número;</li>
    <li>Convertendo todos registros da feature line_id de object para int64;</li>
    <li>Sequenciamento em ordem crescente dos registros, tendo por base a feature line_id;</li>
    <li>Checagem de registros duplicados na feature line_id e remoção;</li>
    <li>Merge dos datasets: Mês de Janeiro de 2023 - iniciando no dia 01 e finalizando do dia 31; </li>    
    <li>Merge dos datasets: Mês de Junho de 2024 - iniciando no dia 05 e finalizando do dia 30; </li>
    <li>Carregando os dois datasets mergiados;</li>    
    <li>Concatenação dos datasets mergiados visando a geração de um único dataset.</li>
</ul>

In [7]:
for day in days_list:
    CAPTURE_DATE = f'{YEAR}_{MONTH}_{day}'
    
    print('Carregando datasets')
    
    path = f"{PATH_ORIGINALS_DATASETS}{CAPTURE_DATE}_"
          
    linhas_df = load_dataset(f'{path}linhas.json', 'json')
    pontos_linha_df = load_dataset(f'{path}pontosLinha.json', 'json')
    veiculos_df = load_dataset(f'{path}tabelaVeiculo.json', 'json')
    tabela_linha_df = load_dataset(f'{path}tabelaLinha.json', 'json')
    trechos_itinerarios_df = load_dataset(f'{path}trechosItinerarios.json', 'json')
        
    print('Datasets carregados com sucesso!')

    print('--'*60)
    
    line_df = linhas_df.copy()
    
    line_df_renamed_columns = {
        'COD': 'line_id',
        'NOME': 'line_name',
        'NOME_COR': 'line_color',
        'CATEGORIA_SERVICO': 'line_category',
        'SOMENTE_CARTAO': 'line_payment_method'
    }
    print(f'Iniciando pre-processamento dataset line_df - data: {CAPTURE_DATE}')
    rename_features(line_df, line_df_renamed_columns, True, False)
    validate_numeric_registers(line_df, 'line_id', False)
    line_df['line_id'] = clean_non_numeric_registers(line_df, 'line_id', False)
    validate_numeric_registers(line_df, 'line_id', False)
    line_df['line_id'] = convert_object_to_int(line_df, 'line_id', False)
    line_df = sequencing_registers(line_df, 'line_id', False)
    checks_duplicate_registers(line_df, 'line_id', False)
    show_duplicate_registers(line_df, 'line_id', False)
    line_df = remove_duplicate_registers(line_df, 'line_id', False)
    checks_duplicate_registers(line_df, 'line_id', False)
    print(f'Finalizando pre-processamento dataset line_df - data: {CAPTURE_DATE}')
    
    print('--'*60)
    
    station_df = pontos_linha_df.copy()
    
    station_df_renamed_columns = {
        'NOME': 'station_name',
        'NUM': 'station_code',
        'LAT': 'station_initial_latitude',
        'LON': 'station_initial_longitude',
        'SEQ': 'station_sequence',
        'SENTIDO': 'station_direction',
        'TIPO': 'station_type',
        'COD': 'line_id',
    }
    print(f'Iniciando pre-processamento dataset station_df - data: {CAPTURE_DATE}')
    rename_features(station_df, station_df_renamed_columns, True, False)
    remove_features(station_df, ['GRUPO','ITINERARY_ID'], True, False)
    validate_numeric_registers(station_df, 'line_id', False)
    station_df['line_id'] = clean_non_numeric_registers(station_df, 'line_id', False)
    validate_numeric_registers(station_df, 'line_id', False)
    station_df['line_id'] = convert_object_to_int(station_df, 'line_id', False)
    station_df = sequencing_registers(station_df, 'line_id', False)
    checks_duplicate_registers(station_df, 'line_id', False)
    show_duplicate_registers(station_df, 'line_id', False)
    station_df = remove_duplicate_registers(station_df, 'line_id', False)
    checks_duplicate_registers(station_df, 'line_id', False)
    print(f'Finalizando pre-processamento dataset station_df - data: {CAPTURE_DATE}')
    
    print('--'*60)
    
    vehicle_df = veiculos_df.copy()
    
    vehicle_df_renamed_columns = { 
        'VEICULO': 'vehicle_plate', 
        'HORARIO': 'vehicle_time',
        'COD_LINHA': 'line_id'
    }
    
    vehicle_df_columns_to_remove = ['NOME_LINHA','TABELA', 'COD_PONTO']
    
    print(f'Iniciando pre-processamento dataset vehicle_df - data: {CAPTURE_DATE}')
    rename_features(vehicle_df, vehicle_df_renamed_columns, True, False)
    remove_features(vehicle_df, vehicle_df_columns_to_remove, True, False)
    validate_numeric_registers(vehicle_df, 'line_id', False)
    vehicle_df['line_id'] = clean_non_numeric_registers(vehicle_df, 'line_id', False)
    validate_numeric_registers(vehicle_df, 'line_id', False)
    vehicle_df['line_id'] = convert_object_to_int(vehicle_df, 'line_id', False)
    vehicle_df = sequencing_registers(vehicle_df, 'line_id', False)
    checks_duplicate_registers(vehicle_df, 'line_id', False)
    show_duplicate_registers(vehicle_df, 'line_id', False)
    vehicle_df = remove_duplicate_registers(vehicle_df, 'line_id', False)
    checks_duplicate_registers(vehicle_df, 'line_id', False)
    print(f'Finalizando pre-processamento dataset vehicle_df - data: {CAPTURE_DATE}')
    
    print('--'*60)
    
    accessibility_info_df = tabela_linha_df.copy()
    
    accessibility_info_df_renamed_columns = {
        'ADAPT': 'accessibility_info_eletric_ramp',
        'HORA': 'accessibility_info_time',
        'DIA': 'accessibility_info_day',
        'COD': 'line_id'
    }
    accessibility_info_df_columns_to_remove = ['PONTO','NUM','TABELA']
    
    print(f'Iniciando pre-processamento dataset accessibility_info_df - data: {CAPTURE_DATE}')
    rename_features(accessibility_info_df, accessibility_info_df_renamed_columns, True, False)
    remove_features(accessibility_info_df, accessibility_info_df_columns_to_remove, True, False)
    validate_numeric_registers(accessibility_info_df, 'line_id', False)
    accessibility_info_df['line_id'] = clean_non_numeric_registers(accessibility_info_df, 'line_id', False)
    validate_numeric_registers(accessibility_info_df, 'line_id', False)
    accessibility_info_df['line_id'] = convert_object_to_int(accessibility_info_df, 'line_id', False)
    accessibility_info_df = sequencing_registers(accessibility_info_df, 'line_id', False)
    checks_duplicate_registers(accessibility_info_df, 'line_id', False)
    show_duplicate_registers(accessibility_info_df, 'line_id', False)
    accessibility_info_df = remove_duplicate_registers(accessibility_info_df, 'line_id', False)
    checks_duplicate_registers(accessibility_info_df, 'line_id', False)
    print(f'Finalizando pre-processamento dataset accessibility_info_df - data: {CAPTURE_DATE}')
    
    print('--'*60)
    
    itinerary_df = trechos_itinerarios_df.copy()
    
    itinerary_df_renamed_columns = { 
        'NOME_EMPRESA': 'itinerary_company_name',
        'COD_ITINERARIO': 'itinerary_code',
        'PTO_ESPECIAL': 'itinerary_special_line',
        'COD_LINHA': 'line_id',
    }
    
    itinerary_df_columns_to_remove = [
        'NOME_LINHA',
        'COD_CATEGORIA',
        'NOME_CATEGORIA',
        'COD_EMPRESA',
        'COD_PTO_PARADA_TH',
        'NOME_PTO_PARADA_TH',
        'NOME_PTO_ABREVIADO',
        'SEQ_PTO_ITI_TH',
        'NOME_ITINERARIO',
        'COD_PTO_TRECHO_A',
        'SEQ_PONTO_TRECHO_A',
        'COD_PTO_TRECHO_B',
        'SEQ_PONTO_TRECHO_B',
        'EXTENSAO_TRECHO_A_ATE_B',
        'TIPO_TRECHO',
        'STOP_CODE',
        'STOP_NAME',
        'CODIGO_URBS'
    ]
    
    print(f'Iniciando pre-processamento dataset itinerary_df - data: {CAPTURE_DATE}')
    rename_features(itinerary_df, itinerary_df_renamed_columns, True, False)
    remove_features(itinerary_df, itinerary_df_columns_to_remove, True, False)
    validate_numeric_registers(itinerary_df, 'line_id', False)
    itinerary_df['line_id'] = clean_non_numeric_registers(itinerary_df, 'line_id', False)
    validate_numeric_registers(itinerary_df, 'line_id', False)
    itinerary_df['line_id'] = convert_object_to_int(itinerary_df, 'line_id', False)
    itinerary_df = sequencing_registers(itinerary_df, 'line_id', False)
    checks_duplicate_registers(itinerary_df, 'line_id', False)
    show_duplicate_registers(itinerary_df, 'line_id', False)
    itinerary_df = remove_duplicate_registers(itinerary_df, 'line_id', False)
    checks_duplicate_registers(itinerary_df, 'line_id', False)
    print(f'Finalizando pre-processamento dataset itinerary_df - data: {CAPTURE_DATE}')
    
    print('--'*60)
    
    print(f'Iniciando merge de todos datasets inicialmente pre-processados - data: {CAPTURE_DATE}')
    line_station_df = pd.merge(line_df, station_df, on='line_id')
    line_station_vehicle_df = pd.merge(line_station_df, vehicle_df, on='line_id')
    line_station_vehicle_accessibility_info_df = pd.merge(line_station_vehicle_df, accessibility_info_df, on='line_id')
    line_station_vehicle_accessibility_info_itinerary_df = pd.merge(line_station_vehicle_accessibility_info_df, itinerary_df, on='line_id')
    df = line_station_vehicle_accessibility_info_itinerary_df
    print(f'Merge finalizado - data: {CAPTURE_DATE}')
    
    print('--'*60)
    
    print(f'Salvando dataset - data: {YEAR}_{MONTH}_{day}')
    dataset_file_name = f'merged_dataset_{YEAR}_{MONTH}_{day}_.csv'
    df.to_csv(f'{PATH_INITIAL_PREPROCESSING_DATASETS}{dataset_file_name}', index=False)
    print('Dataset salvo com sucesso')

Carregando datasets
Datasets carregados com sucesso!
------------------------------------------------------------------------------------------------------------------------
Iniciando pre-processamento dataset line_df - data: 2024_06_05
Finalizando pre-processamento dataset line_df - data: 2024_06_05
------------------------------------------------------------------------------------------------------------------------
Iniciando pre-processamento dataset station_df - data: 2024_06_05
Finalizando pre-processamento dataset station_df - data: 2024_06_05
------------------------------------------------------------------------------------------------------------------------
Iniciando pre-processamento dataset vehicle_df - data: 2024_06_05
Finalizando pre-processamento dataset vehicle_df - data: 2024_06_05
------------------------------------------------------------------------------------------------------------------------
Iniciando pre-processamento dataset accessibility_info_df - data: 2

<h4><b>Concatenando datasets mergiados</b></h4>

In [12]:
df_list = []
for day in days_list:
    df = pd.read_csv(f'{PATH_INITIAL_PREPROCESSING_DATASETS}merged_dataset_{YEAR}_{MONTH}_{day}_.csv')
    df_list.append(df)
    
df_concat = pd.concat(df_list, ignore_index=True)
df_concat.to_csv(f'{PATH_CONCATENATED_DATASETS}concatenated_dataset_{YEAR}_{MONTH}.csv', index=False)

<h4><b>Reunindo datasets pré-processados, mergiados e concatenados visando a geração de um único dataset.</b></h4>
<p>Geração de um dataset único, formado pela junção de todos datasets.</p>

In [18]:
df_concatenated_dataset_list = []
month_list = [
    {'year': '2023', 'month': '01'},
    {'year': '2024', 'month': '06'}
]

for month in month_list:
    df_concatenated = pd.read_csv(f'{PATH_CONCATENATED_DATASETS}concatenated_dataset_{month["year"]}_{month["month"]}.csv')
    df_concatenated_dataset_list.append(df_concatenated)

df_joined = pd.concat(df_concatenated_dataset_list, ignore_index=True)
df_joined.to_csv(f'{PATH_JOINED_DATASETS}joined_dataset.csv', index=False)

In [21]:
df = load_dataset(f'{PATH_JOINED_DATASETS}joined_dataset.csv', 'csv')